In [ ]:
'''
Requirements:
US Bureau of Labor Statistic API Key in config.py
Postgres credentials in config.py

Sample Code Source:
https://www.bls.gov/developers/api_python.htm#python2

Data website:
https://www.kroger.com/p/smithfield-thick-cut-naturally-hickory-smoked-bacon/0007080004125
https://data.bls.gov/timeseries/APU0000704111
'''
;

In [5]:
# Import Modules
import pandas as pd
import requests
import json
import datetime as dt

from splinter import Browser
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from config import bls_api_key, username, passphrase

# Kroger Smithfield Thick Cut Naturally Hickory Smoked Bacon 16oz pricing data

In [4]:
# Kroger Pricing ETL

# Splinter browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Read url
url = 'https://www.kroger.com/p/smithfield-thick-cut-naturally-hickory-smoked-bacon/0007080004125'
browser.visit(url)

# Browse and look for confirm and click thru
browser.find_by_text('Confirm').first.click()

# Scrape website and look for price of bacon
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
quotes = soup.find_all('mark',  class_="kds-Price-promotional")

# Once price is find, split and just collect the price
price=str(quotes[0]).split('>')[1].split('<')[0]
price=price.replace("$","")
print(price)

# Find the date
my_date=dt.date.today()

# Store the price and date into a dictionary
data_dict={"date":[my_date],
          "price":[price]}

# Print out data_dict
data_dict

# Convert into a DataFrame
data_pd=pd.DataFrame(data_dict)

# Show DataFrame
data_pd

# Connect to Postgres
rds_connection_string = f"{username}:{passphrase}@localhost:5432/bacon_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Create table to PG Admin
data_pd.to_sql(name='kroger_price', con=engine, if_exists='append', index=False)

# Close the brower
browser.quit()

3.50


# US Bureau of Labor Statistics Bacon CPI Data

In [6]:
now_year = dt.date.today().year

def download_bls_api(startyear =  now_year - 1, endyear = now_year):
    
    '''Default to since last year'''

    # Define variables
    series_id = 'APU0000704111'
    
    # Convert years to string
    startyear = str(startyear)
    endyear   = str(endyear)

    # Pulling API Data from US Bureau of Labor Statistics
    headers = {'Content-type': 'application/json'}
    data    = json.dumps({"seriesid": [series_id], "startyear"  : startyear, "endyear" : endyear, "registrationkey": bls_api_key})
    p       = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = data, headers = headers)
    json_data = json.loads(p.text)
    
    # Print Status
    print(json_data["status"])
    
    # Print API return message if it's not empty
    if json_data['message']:
        print(json_data['message'])
        
    return json_data

def transform_bls_data(json_data):

    # Create a pandas DataFrame
    data_dict = {
        'year_month'  : [],
        'value' : [],
    }

    for item in json_data['Results']['series'][0]['data']:

        year_month  = item['year'] + item['period'].replace('M', '')
        value  = float(item['value'])
        
        data_dict['year_month'].append(year_month)
        data_dict['value'].append(value)

    data_pd = pd.DataFrame(data_dict)

    return data_pd

def store_cpi_data_to_db(data_pd):

    # Connect to Postgres
    rds_connection_string = f"{username}:{passphrase}@localhost:5432/bacon_db"
    engine = create_engine(f'postgresql://{rds_connection_string}')

    # Pull the last 24 data points from the database
    try:
        last_24m = pd.read_sql_query('SELECT * FROM bacon_cpi LIMIT 24', con=engine)
    except:
        # If the table bacon_cpi doesn't exsit in the database. Create one.
        data_pd.to_sql(name='bacon_cpi', con = engine, if_exists='append', index = False)
        
        print('A new table bacon_cpi has been created in the database bacon_db.')
        return
    
    # Store only the new data to the database
    for year_month in data_pd.year_month:
        if year_month not in last_24m.year_month.values:
        
            new_data = data_pd.loc[data_pd.year_month == year_month].copy()
        
            # Load Bacon CPI Data to Postgres
            new_data.to_sql(name='bacon_cpi', con = engine, if_exists='append', index = False)
        
            print(year_month, 'cpi data has been loaded to the database bacon_db.')
            
    return

# Extract Data
json_data = download_bls_api()

# Transform data
data_pd = transform_bls_data(json_data)

# Load data
store_cpi_data_to_db(data_pd)

REQUEST_SUCCEEDED
A new table bacon_cpi has been created in the database bacon_db.


# Lean Hogs futures from Investing.com.

In [10]:
def lean_hog_data_pull():
    # Creating the browser to scrape with
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    
    # Giving the web path to the site we are scraping
    url = 'https://www.investing.com/commodities/lean-hogs-historical-data'
    browser.visit(url)
    
    # Iterate through all pages
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Finding the correct table, there are multiple tables on the page, ours is the only with "Open" in the table
    texts = soup.find_all("table")
    for item in texts:
        if "Open" in item.get_text():
            tabletext = item.get_text()
            
    # Cleaning up some of the text to prepare for a list of lists
    tabletext = tabletext.replace("\n", "|")
    tabletext = tabletext.replace("|-", "|0.00K")
    tabletext = tabletext.replace("Vol.", "Volume|")
    tabletext = tabletext.replace("K", "K|")
    tabletext = tabletext.replace("%", "PCT")
    tabletext = tabletext.replace(" Change", "Change")
    
    # Splitting the data to create the first list; cleaning it up a bit
    tablelist = tabletext.split("|||")
    cleanerlist = []
    for item in tablelist:
        if item != "":
            cleanerlist.append(item.replace("||",""))
            
    # Creating a list of lists
    listolist = []
    for item in tablelist:
        listolist.append(item.split("|"))
    
    #Removing blank lists
    listolist = [x for x in listolist if x != ['']]
    
    #Creating a clean list of lists where each list has the blanks ('') removed
    clistolist = []
    for item in listolist:
        clistolist.append(list(filter(None,item)))
    for item in clistolist:
        for itemtwo in item:
            itemtwo = itemtwo.strip()
            
    #Turning the clean list of lists into a dataframe
    df = pd.DataFrame(clistolist[1:], columns = clistolist[0])
    
    #Cleaning up the dataframe
    df['Volume'] = df['Volume'].str.replace('K','')
    df['Date'] = df['Date'].str.replace(", ","-")
    df['Date'] = df['Date'].str.replace("Jan ","01-")
    df['Date'] = df['Date'].str.replace("Feb ","02-")
    df['Date'] = df['Date'].str.replace("Mar ","03-")
    df['Date'] = df['Date'].str.replace("Apr ","04-")
    df['Date'] = df['Date'].str.replace("May ","05-")
    df['Date'] = df['Date'].str.replace("Jun ","06-")
    df['Date'] = df['Date'].str.replace("Jul ","07-")
    df['Date'] = df['Date'].str.replace("Aug ","08-")
    df['Date'] = df['Date'].str.replace("Sep ","09-")
    df['Date'] = df['Date'].str.replace("Oct ","10-")
    df['Date'] = df['Date'].str.replace("Nov ","11-")
    df['Date'] = df['Date'].str.replace("Dec ","12-")
    df['Change PCT'] = df['Change PCT'].str.replace("PCT","")
    df['Change PCT'] = pd.to_numeric(df['Change PCT']) / 100
    df['Volume'] = pd.to_numeric(df['Volume']) * 1000
    df['Price'] = pd.to_numeric(df['Price'])
    df['Open'] = pd.to_numeric(df['Open'])
    df['High'] = pd.to_numeric(df['High'])
    df['Low'] = pd.to_numeric(df['Low'])
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Connect to Postgres
    rds_connection_string = f"{username}:{passphrase}@localhost:5432/bacon_db"
    engine = create_engine(f'postgresql://{rds_connection_string}')
    
    # Pulling the historical data if it exists
    try:
        olddf = pd.read_sql_table("lean_hog_commodity", con=engine)
        fulldf = olddf.append(df,ignore_index=True)
    except:
        fulldf = df
    fulldf = fulldf.drop_duplicates('Date',keep='last')
    
    # Load Lean Hog Commodity to Postgres
    fulldf.to_sql(name='lean_hog_commodity', con = engine, if_exists='replace', index = False)
    
    # Closing out the browser
    browser.quit()
    return fulldf

fulldf = lean_hog_data_pull()